# ****FLUSHING ANALYTICS****


# Installing Dependencies

In [1]:
# %pip install pandas
# %pip install pyproj
# %pip install scikit-learn
# %pip install openpyxl
# %pip install matplotlib
# %pip install seaborn

In [2]:
# %pip install xgboost
# %pip install optuna

# Preparing the Data

## Loading and exploring the datasets

In [3]:
# Importing dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from pyproj import Proj, transform

# file_path = '/EllaTesting/Files/Repairs_2018_2023.xlsx'

# print(file_path)

# Creating DataFrames from spreadsheets
all_hydrants_filepath = "Qry_AllHY.csv"
flushing_history_filepath = "Qry_FlushHistory.csv"

all_hydrants_df = pd.read_csv(all_hydrants_filepath)
flushing_history_df = pd.read_csv(flushing_history_filepath)

C:\Users\26154\AppData\Local\Temp\1\ipykernel_17448\881486883.py:20: DtypeWarning: Columns (33,50,57,91) have mixed types. Specify dtype option on import or set low_memory=False.
  all_hydrants_df = pd.read_csv(all_hydrants_filepath)
C:\Users\26154\AppData\Local\Temp\1\ipykernel_17448\881486883.py:21: DtypeWarning: Columns (0,4,5,15,23,24,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  flushing_history_df = pd.read_csv(flushing_history_filepath)


## Data Cleaning and Preprocessing

### Data Cleaning - History Table

In [4]:
# Creating a copy of the Flushing History DataFrame
copied_flushing_history_df = copy.deepcopy(flushing_history_df)

In [5]:
#delete all unnecessary columns
copied_flushing_history_df = copied_flushing_history_df.drop(columns={'FLSHTIME3', 'FLSHTIME4', 'FLSHTIME5', 'FLSHTIME6', 'FLSHTIME7', 'FLSHTIME8', 'FLSHTIME9', 'FLSHTIME10', 'FLSHTIME11', 'FLSHVOL1', 'FLSHVOL2', 'FLSHVOL3', 'FLSHVOL4', 'FLSHVOL5', 'FLSHVOL6', 'FLSHVOL7', 'FLSHVOL8', 'FLSHVOL9', 'FLSHVOL10', 'FLSHVOL11', 'CLFREE3', 
                                                                      'CLFREE4', 'CLFREE5', 'CLFREE6', 'CLFREE7', 'CLFREE8', 'CLFREE9', 'CLFREE10', 'CLFREE11', 'CLTOTL3', 'CLTOTL4', 'CLTOTL5', 'CLTOTL6', 'CLTOTL7', 'CLTOTL8', 'CLTOTL9', 'CLTOTL10', 'CLTOTL11', 'ADDBY', 'ADDDTTM', 'MODBY', 'MODDTTM', 'REMARKS', 'FLSHTIME', 'FLUSHNC', 'NFREASON',  
                                                                      'CLTOTL1', 'CLTOTL2', 'CLTOTL', 'UNSCEHDLD',  'ALTERNATE'})

#### Inspecting the data

In [6]:
copied_flushing_history_df.shape

(116808, 16)

In [7]:
copied_flushing_history_df = copied_flushing_history_df.dropna(subset=['FLSHFRE', 'FLSHDATE', 'CLFREE', 'PHREAD', 'FLSHVOL', 'FLSHDUR', 'COMPKEY'])

In [8]:
copied_flushing_history_df.shape

(112606, 16)

In [9]:
copied_flushing_history_df.head(10)

,INSPKEY,GRIDKEY,FLSHFRE,FLSHDATE,CLFREE,PHREAD,FLSHVOL,FLSHDUR,INSPBY,CLFREE1,CLFREE2,WWNUM,COMMUN,COMPKEY,FLSHTIME1,FLSHTIME2
6,965966,19046430,1 WEEK,2019-04-23,1.18,7.92,1.000,5.0,61535,NaN,NaN,WW220003993,CD,805425.0,NaN,NaN
7,972209,19048814,1 MONTH,2019-04-25,1.66,7.90,18.000,10.0,65673,1.67,NaN,WW220003993,CV,9876.0,15:05,NaN
8,972253,19048815,1 MONTH,2019-04-25,1.33,7.51,10.000,10.0,65673,1.30,NaN,WW260002590,CR,540642.0,09:55,NaN
9,965951,19048816,1 WEEK,2019-04-29,1.69,7.50,6.000,5.0,65673,NaN,NaN,WW260002590,CT,531455.0,NaN,NaN
10,965955,19048817,2 WEEKS,2019-04-30,1.67,7.86,10.000,10.0,65673,1.64,NaN,WW220003993,CV,541072.0,10:50,NaN
11,965950,19048818,1 WEEK,2019-04-30,0.72,8.09,23.000,15.0,65673,0.77,0.82,WW260094120,CB,528297.0,09:40,09:45
12,970581,19048819,2 WEEKS,2019-05-01,1.09,8.21,1.000,5.0,61535,NaN,NaN,WW260094120,CB,9986.0,NaN,NaN
13,970575,19048820,2 WEEKS,2019-05-01,1.11,8.31,1.000,5.0,61535,NaN,NaN,WW260094120,CCC,9263.0,NaN,NaN
14,965730,19048821,3 DAYS,2019-03-01,0.92,7.69,1.194,20.0,35832,NaN,NaN,WW260094120,B,7781.0,NaN,NaN
15,965730,19048822,3 DAYS,2019-04-30,0.42,7.89,5.000,13.0,68527,0.71,NaN,WW260094120,B,7781.0,07:40,NaN


In [10]:
copied_flushing_history_df['CLFREE2'] = copied_flushing_history_df['CLFREE2'].fillna(0)

In [11]:
copied_flushing_history_df['CLFREEFINAL'] = np.where(copied_flushing_history_df['CLFREE2'] > 0, copied_flushing_history_df['CLFREE2'], copied_flushing_history_df['CLFREE1'])

In [12]:
copied_flushing_history_df = copied_flushing_history_df.dropna(subset=['CLFREEFINAL'])
copied_flushing_history_df

,INSPKEY,GRIDKEY,FLSHFRE,FLSHDATE,CLFREE,PHREAD,FLSHVOL,FLSHDUR,INSPBY,CLFREE1,CLFREE2,WWNUM,COMMUN,COMPKEY,FLSHTIME1,FLSHTIME2,CLFREEFINAL
7,972209,19048814,1 MONTH,2019-04-25,1.66,7.90,18.0,10.0,65673,1.67,0.00,WW220003993,CV,9876.0,15:05,NaN,1.67
8,972253,19048815,1 MONTH,2019-04-25,1.33,7.51,10.0,10.0,65673,1.30,0.00,WW260002590,CR,540642.0,09:55,NaN,1.30
10,965955,19048817,2 WEEKS,2019-04-30,1.67,7.86,10.0,10.0,65673,1.64,0.00,WW220003993,CV,541072.0,10:50,NaN,1.64
11,965950,19048818,1 WEEK,2019-04-30,0.72,8.09,23.0,15.0,65673,0.77,0.82,WW260094120,CB,528297.0,09:40,09:45,0.82
15,965730,19048822,3 DAYS,2019-04-30,0.42,7.89,5.0,13.0,68527,0.71,0.00,WW260094120,B,7781.0,07:40,NaN,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116802,1242027,19513855,3 DAYS,2024-04-30,1.19,7.30,5.0,15.0,17571,1.23,0.00,WW260094120,B,796912.0,11:25,NaN,1.23
116803,1242028,19513856,3 DAYS,2024-04-15,1.00,8.25,13.0,10.0,13394,0.98,0.00,WW260094120,CDR,798643.0,08:30,NaN,0.98
116805,1242029,19513858,2 DAYS,2024-05-01,0.66,7.82,26.0,20.0,13787,1.01,0.00,WW260094120,M,798849.0,09:50,NaN,1.01
116806,1242628,19513859,2 WEEKS,2024-05-02,1.17,7.98,20.0,15.0,68870,1.10,0.00,WW260094120,B,801724.0,13:50,NaN,1.10


In [13]:
copied_flushing_history_df = copied_flushing_history_df.drop(columns=['CLFREE1', 'CLFREE2', 'FLSHTIME1', 'FLSHTIME2', 'INSPKEY', 'GRIDKEY', 'FLSHVOL', 'FLSHDUR', 'INSPBY', 'COMMUN'])

In [14]:
copied_flushing_history_df

,FLSHFRE,FLSHDATE,CLFREE,PHREAD,WWNUM,COMPKEY,CLFREEFINAL
7,1 MONTH,2019-04-25,1.66,7.90,WW220003993,9876.0,1.67
8,1 MONTH,2019-04-25,1.33,7.51,WW260002590,540642.0,1.30
10,2 WEEKS,2019-04-30,1.67,7.86,WW220003993,541072.0,1.64
11,1 WEEK,2019-04-30,0.72,8.09,WW260094120,528297.0,0.82
15,3 DAYS,2019-04-30,0.42,7.89,WW260094120,7781.0,0.71
...,...,...,...,...,...,...,...
116802,3 DAYS,2024-04-30,1.19,7.30,WW260094120,796912.0,1.23
116803,3 DAYS,2024-04-15,1.00,8.25,WW260094120,798643.0,0.98
116805,2 DAYS,2024-05-01,0.66,7.82,WW260094120,798849.0,1.01
116806,2 WEEKS,2024-05-02,1.17,7.98,WW260094120,801724.0,1.10


In [15]:
copied_flushing_history_df = copied_flushing_history_df.drop(copied_flushing_history_df[copied_flushing_history_df['PHREAD'] > 2000].index)
copied_flushing_history_df

,FLSHFRE,FLSHDATE,CLFREE,PHREAD,WWNUM,COMPKEY,CLFREEFINAL
7,1 MONTH,2019-04-25,1.66,7.90,WW220003993,9876.0,1.67
8,1 MONTH,2019-04-25,1.33,7.51,WW260002590,540642.0,1.30
10,2 WEEKS,2019-04-30,1.67,7.86,WW220003993,541072.0,1.64
11,1 WEEK,2019-04-30,0.72,8.09,WW260094120,528297.0,0.82
15,3 DAYS,2019-04-30,0.42,7.89,WW260094120,7781.0,0.71
...,...,...,...,...,...,...,...
116802,3 DAYS,2024-04-30,1.19,7.30,WW260094120,796912.0,1.23
116803,3 DAYS,2024-04-15,1.00,8.25,WW260094120,798643.0,0.98
116805,2 DAYS,2024-05-01,0.66,7.82,WW260094120,798849.0,1.01
116806,2 WEEKS,2024-05-02,1.17,7.98,WW260094120,801724.0,1.10


In [16]:
copied_flushing_history_df = copied_flushing_history_df.drop(copied_flushing_history_df[copied_flushing_history_df['PHREAD'] < 0].index)
copied_flushing_history_df

,FLSHFRE,FLSHDATE,CLFREE,PHREAD,WWNUM,COMPKEY,CLFREEFINAL
7,1 MONTH,2019-04-25,1.66,7.90,WW220003993,9876.0,1.67
8,1 MONTH,2019-04-25,1.33,7.51,WW260002590,540642.0,1.30
10,2 WEEKS,2019-04-30,1.67,7.86,WW220003993,541072.0,1.64
11,1 WEEK,2019-04-30,0.72,8.09,WW260094120,528297.0,0.82
15,3 DAYS,2019-04-30,0.42,7.89,WW260094120,7781.0,0.71
...,...,...,...,...,...,...,...
116802,3 DAYS,2024-04-30,1.19,7.30,WW260094120,796912.0,1.23
116803,3 DAYS,2024-04-15,1.00,8.25,WW260094120,798643.0,0.98
116805,2 DAYS,2024-05-01,0.66,7.82,WW260094120,798849.0,1.01
116806,2 WEEKS,2024-05-02,1.17,7.98,WW260094120,801724.0,1.10


In [17]:
copied_flushing_history_df = copied_flushing_history_df.drop(copied_flushing_history_df[copied_flushing_history_df['CLFREE'] == 0].index)
copied_flushing_history_df

,FLSHFRE,FLSHDATE,CLFREE,PHREAD,WWNUM,COMPKEY,CLFREEFINAL
7,1 MONTH,2019-04-25,1.66,7.90,WW220003993,9876.0,1.67
8,1 MONTH,2019-04-25,1.33,7.51,WW260002590,540642.0,1.30
10,2 WEEKS,2019-04-30,1.67,7.86,WW220003993,541072.0,1.64
11,1 WEEK,2019-04-30,0.72,8.09,WW260094120,528297.0,0.82
15,3 DAYS,2019-04-30,0.42,7.89,WW260094120,7781.0,0.71
...,...,...,...,...,...,...,...
116802,3 DAYS,2024-04-30,1.19,7.30,WW260094120,796912.0,1.23
116803,3 DAYS,2024-04-15,1.00,8.25,WW260094120,798643.0,0.98
116805,2 DAYS,2024-05-01,0.66,7.82,WW260094120,798849.0,1.01
116806,2 WEEKS,2024-05-02,1.17,7.98,WW260094120,801724.0,1.10


In [18]:
copied_flushing_history_df = copied_flushing_history_df.drop(copied_flushing_history_df[copied_flushing_history_df['CLFREEFINAL'] == 0].index)
copied_flushing_history_df

,FLSHFRE,FLSHDATE,CLFREE,PHREAD,WWNUM,COMPKEY,CLFREEFINAL
7,1 MONTH,2019-04-25,1.66,7.90,WW220003993,9876.0,1.67
8,1 MONTH,2019-04-25,1.33,7.51,WW260002590,540642.0,1.30
10,2 WEEKS,2019-04-30,1.67,7.86,WW220003993,541072.0,1.64
11,1 WEEK,2019-04-30,0.72,8.09,WW260094120,528297.0,0.82
15,3 DAYS,2019-04-30,0.42,7.89,WW260094120,7781.0,0.71
...,...,...,...,...,...,...,...
116802,3 DAYS,2024-04-30,1.19,7.30,WW260094120,796912.0,1.23
116803,3 DAYS,2024-04-15,1.00,8.25,WW260094120,798643.0,0.98
116805,2 DAYS,2024-05-01,0.66,7.82,WW260094120,798849.0,1.01
116806,2 WEEKS,2024-05-02,1.17,7.98,WW260094120,801724.0,1.10


In [19]:
copied_flushing_history_df['CLFREE'] = copied_flushing_history_df['CLFREE'].replace(78, 0.78)
copied_flushing_history_df

,FLSHFRE,FLSHDATE,CLFREE,PHREAD,WWNUM,COMPKEY,CLFREEFINAL
7,1 MONTH,2019-04-25,1.66,7.90,WW220003993,9876.0,1.67
8,1 MONTH,2019-04-25,1.33,7.51,WW260002590,540642.0,1.30
10,2 WEEKS,2019-04-30,1.67,7.86,WW220003993,541072.0,1.64
11,1 WEEK,2019-04-30,0.72,8.09,WW260094120,528297.0,0.82
15,3 DAYS,2019-04-30,0.42,7.89,WW260094120,7781.0,0.71
...,...,...,...,...,...,...,...
116802,3 DAYS,2024-04-30,1.19,7.30,WW260094120,796912.0,1.23
116803,3 DAYS,2024-04-15,1.00,8.25,WW260094120,798643.0,0.98
116805,2 DAYS,2024-05-01,0.66,7.82,WW260094120,798849.0,1.01
116806,2 WEEKS,2024-05-02,1.17,7.98,WW260094120,801724.0,1.10


In [20]:
# Calculates a new column representing the difference between the CLFREE reading before a flush and the CLFREEFINAL reading of the previous flush (the first flush for a given hydrant is given the value equal to the CLFREEFINAL reading of the flush)
copied_flushing_history_df = copied_flushing_history_df.sort_values('COMPKEY')

copied_flushing_history_df['CLDELTA'] = copied_flushing_history_df.groupby('COMPKEY')['CLFREEFINAL'].shift(1)  # shift CLFREEFINAL up by 1 row within each COMPKEY group

copied_flushing_history_df['CLDELTA'] = copied_flushing_history_df['CLFREE'] - copied_flushing_history_df['CLDELTA']  # calculate the difference between CLFREE and shifted CLFREEFINAL

copied_flushing_history_df.loc[copied_flushing_history_df['CLDELTA'].isnull(), 'CLDELTA'] = copied_flushing_history_df['CLFREEFINAL']  # fill NaN values with CLFREEFINAL (for the first row of each COMPKEY group)

In [21]:
wwnum_temp = pd.get_dummies(copied_flushing_history_df['WWNUM'])
copied_flushing_history_df = copied_flushing_history_df.join(wwnum_temp)

In [22]:
copied_flushing_history_df

,FLSHFRE,FLSHDATE,CLFREE,PHREAD,WWNUM,COMPKEY,CLFREEFINAL,CLDELTA,WW220003993,WW220004000,WW220004037,WW260002590,WW260094120
98795,1 YEAR,2022-11-24,0.92,8.06,WW260094120,1002.0,0.96,0.96,False,False,False,False,True
85374,1 YEAR,2021-11-24,0.87,8.02,WW260094120,1002.0,1.23,-0.09,False,False,False,False,True
15528,1 YEAR,2015-11-30,0.67,0.00,WW260094120,1002.0,0.81,-0.56,False,False,False,False,True
111638,1 YEAR,2023-11-24,1.08,7.78,WW260094120,1002.0,1.06,0.27,False,False,False,False,True
21057,1 YEAR,2016-11-24,0.70,0.00,WW260094120,1002.0,0.93,-0.36,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115140,1 WEEK,2024-05-30,1.00,7.68,WW220003993,1307858.0,1.16,-0.29,True,False,False,False,False
116030,1 WEEK,2024-05-07,1.27,7.51,WW220003993,1307858.0,1.45,0.11,True,False,False,False,False
116406,1 WEEK,2024-05-14,1.15,7.64,WW220003993,1307858.0,1.24,-0.30,True,False,False,False,False
115821,1 WEEK,2024-06-14,0.69,7.71,WW220003993,1307858.0,0.99,-0.55,True,False,False,False,False


In [23]:
copied_flushing_history_df = copied_flushing_history_df.drop(columns=['WWNUM'])

In [25]:
copied_flushing_history_df.to_excel('historycheck.xlsx', index=False)

**Imputing Missing Categorical Values**

In [ ]:
# #TODO: Ayan: Use the COMPKEY to fill out the missing values in COMMUN
# # Sorting the DataFrame by COMPKEY, as there is a pattern between COMPKEY values and COMMUN values
# copied_flushing_history_df = copied_flushing_history_df.sort_values('COMPKEY')

# # Imputing every missing value with the value that came before it in the sorted DataFrame
# copied_flushing_history_df['COMMUN'] = copied_flushing_history_df['COMMUN'].ffill()

**Imputing Missing Numerical Values**

In [ ]:
# # Imputing missing values in the FLSHVOL and FLSHDUR columns with the median of the column
# copied_flushing_history_df['FLSHVOL'] = copied_flushing_history_df['FLSHVOL'].fillna(copied_flushing_history_df['FLSHVOL'].median())
# copied_flushing_history_df['FLSHDUR'] = copied_flushing_history_df['FLSHDUR'].fillna(copied_flushing_history_df['FLSHDUR'].median())

**Detecting and Replacing Outliers**

In [ ]:
# For visualizing the box and whisker plot
import seaborn as sns

# Replaces all outliers within a given column with the median of that column
def replace_outliers(df, column):
    # Creates a box and whisker plot to visualize data points that are considered to be outliers
    sns.boxplot(df[column])
    plt.title(f"Original Box and Whisker Plot for {column}")
    plt.show()
    
    # Computing the first, second (median), and third quartiles
    q1 = df[column].quantile(0.25)
    q2 = df[column].quantile(0.5)
    q3 = df[column].quantile(0.75)

    print(f"First quartile is {q1}")
    print(f"Median is {q2}")
    print(f"Third quartile is {q3}")
    
    # Computing the interquartile range
    iqr = q3 - q1

    # Values that are lower than the lower fence or higher than the upper fence are considered to be outliers
    lower_fence = q1 - (1.5 * iqr)
    upper_fence = q3 + (1.5 * iqr)

    print(f"Lower fence is {lower_fence}")
    print(f"Upper fence is {upper_fence}")

    # Replace outliers with the median
    adjusted_column = [x if lower_fence <= x <= upper_fence else q2 for x in df[column]]
    
    # New box and whisker plot with outliers adjusted
    sns.boxplot(adjusted_column)
    plt.title(f"New Box and Whisker Plot for {column}")
    plt.show()

    return adjusted_column

In [ ]:
# Replacing outliers in the FLSHVOL column
# copied_flushing_history_df['FLSHVOL'] = replace_outliers(copied_flushing_history_df, 'FLSHVOL')

In [ ]:
# Replacing outliers in the FLSHDUR column
# copied_flushing_history_df['FLSHDUR'] = replace_outliers(copied_flushing_history_df, 'FLSHDUR')

### Data Cleaning - Hydrant Table

In [ ]:
# Creating a copy of the All Hydrants DataFrame
copied_all_hydrants_df = copy.deepcopy(all_hydrants_df)

In [ ]:
# Filters rows where flushing frequency is blank
copied_all_hydrants_df = copied_all_hydrants_df[copied_all_hydrants_df['FLSHFRE'].notnull()]
copied_all_hydrants_df.describe()

In [ ]:
# Dropping unnecessary columns
copied_all_hydrants_df = copied_all_hydrants_df.drop(columns={'ADDRKEY', 'ADDRQUAL', 'AUXVALVE', 'BARRELSIZE', 'COMPLEXKEY', 'GISSTATIC', 'HT', 'INTKEY', 'MAPNO', 'OBST', 'OUTLSZ4', 'PACKING', 'PAINTTYPE', 'PRCLKEY', 'SEGKEY', 'SERNO', 'SLKEY', 'SPECINST', 'STKEY', 'WVKEY', 'BGTNO', 'USGAREAKEY', 'EXPBY', 'EXPDATE', 'ADDBY', 'MODBY', 'MODDTTM',
                                                              'LASTINSP', 'LASTWINT', 'OLDID', 'WMODID', 'MUNICIPAL', 'FLUDATE', 'WWNUM', 'LASTFREE', 'LASTOTAL', 'FLSHFRE_GIS', 'MINFLTM', 'AVKCLIP', 'ASUMDATE', 'MAINTBY', 'REPLABY', 'REPLAFOR', 'FRFLWVLDDT', 'ASSTGRPID', 'EMDEPT', 'EMDIV', 'EMSERV', 'EMSERVCAT', 'EMCAMASTCL', 'EMASSTPRNT', 'EMPURCHDAT',
                                                                  'EMESTSRVLF', 'EMRMGSRVLF', 'EMRMSRLFDT', 'EMREPLCOST', 'EMREPCOSDT', 'EMPURINCST', 'EMASSTCRTL', 'EMASSTCRDT', 'EMASSTRISK', 'EMASTRSKDT', 'EMDISTRICT', 'EMEQPMNTCL', 'EMEQPMNTTP', 'EMASSTDESC', 'EMCOND', 'EMCONDDATE', 'OWN', 'SERVSTAT', 'DISTRICT', 'ASBLT', 'WATPLATT',
                                                                  'LSTFMAX1',	'LSTFMAX2',	'LSTFMAX3',	'AVE3FMAX',	'LSTTMAX1',	'LSTTMAX2',	'LSTTMAX3',	'AVE3TMAX', 'ASBLT', 'ZCOORD', 'PRESSMIN',	'PRESSMAX',	'FIRFLGPM',	'OUTELEVN',	'MDLRESRD', 'FEEDERTYPE'})

In [ ]:
# Filtering out invalid values
# copied_all_hydrants_df = copied_all_hydrants_df[copied_all_hydrants_df['OWN'].isin(['R', 'NAMP'])]
# copied_all_hydrants_df = copied_all_hydrants_df[copied_all_hydrants_df['SERVSTAT'].isin(['A', 'ANYR'])]
# copied_all_hydrants_df = copied_all_hydrants_df[copied_all_hydrants_df['EXPDATE'].isnull()]

**Imputing Missing Categorical Values**

In [ ]:
# Item 3: Clean data, replace nulls with values in some columns
copied_all_hydrants_df['UNITTYPE'] = copied_all_hydrants_df['UNITTYPE'].fillna('STDHY')

In [ ]:
copied_all_hydrants_df['UNITTYPE'].describe()

In [ ]:
#TODO: Ayan: Imputing the missing data for PRESZONE
# Based on other rows with similar data as the missing rows in PRESZONE, PG8 is the most suitable value to impute here
copied_all_hydrants_df['PRESZONE'] = copied_all_hydrants_df['PRESZONE'].fillna("PG8")

In [ ]:
#TODO: Ayan: Imputation for DECLREQD
# Replacing blank values in the 'DECLREQD' column with the mode of the column
copied_all_hydrants_df['DECLREQD'] = copied_all_hydrants_df['DECLREQD'].fillna(copied_all_hydrants_df['DECLREQD'].mode().iloc[0])

In [ ]:
#TODO: Ella: Convert null to N for HYMARKER
copied_all_hydrants_df['HYMARKER'] = copied_all_hydrants_df['HYMARKER'].fillna('N')

In [ ]:
#TODO: Ayan: Impute the missing value for NONSCHE
copied_all_hydrants_df['NONSCHE'] = copied_all_hydrants_df['NONSCHE'].fillna(copied_all_hydrants_df['NONSCHE'].mode().iloc[0])

In [ ]:
copied_all_hydrants_df.head()

**Binning**

In [ ]:
#TODO: Jordan: Imputing the missing data for SUBAREA
#TODO: Jordan: What are the differences between AREA, SUBAREA, COMMUN, WARDID. Which columns to keep and why?

subarea_mode = copied_all_hydrants_df['SUBAREA'].mode()[0]
copied_all_hydrants_df['SUBAREA'] = copied_all_hydrants_df['SUBAREA'].fillna(subarea_mode)
print(copied_all_hydrants_df['SUBAREA'].isna().sum())

# Extracting only the letters from the AREA column
copied_all_hydrants_df['Letters'] = copied_all_hydrants_df['AREA'].str.extract(r'([A-Za-z]+)', expand=False)

# Displaying the resulting DataFrame with only the letters
print(copied_all_hydrants_df[['AREA', 'Letters']])

In [ ]:
copied_all_hydrants_df[copied_all_hydrants_df['Letters'].isnull()]

In [ ]:
# Imputing the missing value with the most suitable value based on 
copied_all_hydrants_df['Letters'] = copied_all_hydrants_df['Letters'].fillna("S")

In [ ]:
# One hot encoding the given columns
copied_all_hydrants_df = pd.get_dummies(data=copied_all_hydrants_df, columns=['COLOR', 'MFGKEY', 'PRESZONE', 'Letters', 'FZONENUM'])

In [ ]:
# Displaying the resulting DataFrame to ensure the columns are included
print("Original DataFrame with extracted letters and one-hot encoded columns:")
print(copied_all_hydrants_df.head())

**Label Encoding**

In [ ]:
# Encoding Y as 1 and N as 0 for the given columns
copied_all_hydrants_df[['LONGKEY', 'PLUGGED', 'NONSCHE', 'DECLREQD', 'HYMARKER']] = (copied_all_hydrants_df[['LONGKEY', 'PLUGGED', 'NONSCHE', 'DECLREQD', 'HYMARKER']] == 'TRUE').astype(int)

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# # One-hot encoding 'COLOR', 'MFGKEY', 'PRESZONE', 'Letters', 'FZONENUM'
# encoder = OneHotEncoder(sparse_output=False)
# encoded_letters = encoder.fit_transform(copied_all_hydrants_df[['Letters']])
# encoded_letters_df = pd.DataFrame(encoded_letters, columns=encoder.get_feature_names_out(['Letters']))

# # Combining the original DataFrame with the encoded letters
# copied_all_hydrants_df = pd.concat([copied_all_hydrants_df, encoded_letters_df], axis=1)

# # Displaying the resulting DataFrame to ensure the columns are included
# print("Original DataFrame with extracted letters and one-hot encoded columns:")
# print(copied_all_hydrants_df.head())

In [ ]:
#TODO: Jordan: Clean up the values in MODEL column (group those that are of similar values but in different forms) & fill the null value with UNKNOWN - Done
#TODO: ELla to clean up the grouping

# # Standardize the values in the MODELNO column
# copied_all_hydrants_df['MODELNO'] = copied_all_hydrants_df['MODELNO'].str.strip().str.upper()

# # Fill missing values with 'UNKNOWN'
# copied_all_hydrants_df['MODELNO'] = copied_all_hydrants_df['MODELNO'].fillna('UNKNOWN')

In [ ]:
# # Define a mapping for similar model numbers
# modelno_mapping = {
#     'BRIGADIER': [
#         'BRIG', 'BRIGADIER', 'BRIGADIER M67', 'BRIGADEER', 'BRIGADERE', 
#         'BRIGD', 'BRIGA', 'BRIGADIER M64', 'BRIGADIER M67B', 
#         'BRIGADIER M67B-002', 'BRIGIDAR', 'BRIGIDERE', 'BRIGG', 'BRIG.', 'brig', 'brig.', 'Brig'
#     ],
#     'CENTURY': [
#         'CENTURY', 'century', 'CEN', 'CENT', 'CENTRY', 'CENTRUY', 
#         'CENURY', 'CENTY', 'CENTRYY', 'CENRTY', 'CCENTURY', 'c entury', 'CEN.'
#     ],
#     '27/100': ['27/100', '27-100', '27-00', '27/00', 'B52700'],
#     'M67': [
#         'M67', 'M67B', 'M67B-002', 'M67-B', 'M67 BRIG', 'M67 BRIGADIER', 
#         'M67 BRIGADIER', 'M67-BRIG', 'M67D', 'M67E', 'M67Y', 'M67m', 
#         'M67\r\nM67', 'M-67', 'M67M', 'M 67', 'M67-brig.', 'm67', 'm67b'
#     ],
#     'D67': ['D67', 'D67M', 'D-67', 'D-67M', 'D67m', 'D57M'],
#     'B50': [
#         'B50', 'B50-B24', 'B50-B18', 'B50-B-24', 'B-50-B', 
#         'B-50', 'B67', 'B560-B24', 'B50B'
#     ],
#     'C': ['C', 'C61', 'C61A'],
#     '3 PORT': [
#         '#5', '5', '3', '3-PORT', '3 PORT', '3 PORTS', '3PORT BRIGADIER', 
#         '3PORT BRIGADIER M67B', '3PORT BRIGADIER M67B', 'M3', 
#         'M-93', 'SIGELOCK SPARTAN', 'THREE PORT', 'TWO PORT'
#     ]
#     # Add more mappings as needed
# }

In [ ]:
# # Reverse the mapping for easy replacement
# reverse_modelno_mapping = {}
# for key, values in modelno_mapping.items():
#     for value in values:
#         reverse_modelno_mapping[value] = key

# # Apply the mapping to the MODELNO column
# copied_all_hydrants_df['MODELNO'] = copied_all_hydrants_df['MODELNO'].replace(reverse_modelno_mapping)

In [ ]:
# # Verify the changes
# print(copied_all_hydrants_df['MODELNO'].unique())
# print(copied_all_hydrants_df['MODELNO'].isna().sum())

# # Save the cleaned dataset
# copied_all_hydrants_df.to_excel('cleaned_hydrantcheck.xlsx', index=False)

In [ ]:
# Bin and then one hot encode the MODELNO column? ***?
# AVK: 27/100, 27 100, 
# MUELLER: 3 PORT, B-05, B20-24, B50, C, CAN-VAL, CENTURY, DARLING,
# CLOW: D67, D-67M, BRIGADIER, M59, M67, MCAVITY, MACAVITY, MUELLER, SENTINEL

**Extracting Dates**

In [ ]:
copied_all_hydrants_df['INSTDATE'] = copied_all_hydrants_df['INSTDATE'].replace('9999-01-01', '2016-03-24')

In [ ]:
# Converting date column to datetime
copied_all_hydrants_df['INSTDATE'] = pd.to_datetime(copied_all_hydrants_df['INSTDATE'], errors='coerce')

# Displaying the resulting DataFrame
print(copied_all_hydrants_df['INSTDATE'].head())

In [ ]:
#from datetime import datetime

# Calculate the age in days
copied_all_hydrants_df['AGE'] = (pd.Timestamp.now() - copied_all_hydrants_df['INSTDATE']).dt.days

# If INSTDATE was missing, AGE will be NaN
copied_all_hydrants_df['AGE'] = copied_all_hydrants_df['AGE'].fillna(-1)  # Filling with -1 to indicate missing ages, you can choose other strategies

print(copied_all_hydrants_df[['INSTDATE', 'AGE']].head(100))

In [ ]:
# Maps a value from FLSHFRE to an integer representing the frequency in days
def convert_to_days(frequency):
    if pd.isna(frequency):
        return None
    frequency = frequency.upper().strip()
    if 'DAY' in frequency:
        return int(frequency.split()[0])
    elif 'DAILY' in frequency:
        return 1
    elif 'WEEK' in frequency:
        return int(frequency.split()[0]) * 7
    elif 'MONTH' in frequency:
        return int(frequency.split()[0]) * 30
    elif 'YEAR' in frequency:
        return int(frequency.split()[0]) * 365
    else:
        return None  # For any unknown frequency format

In [ ]:
# Apply the conversion function
copied_all_hydrants_df['FLSHFRE_DAYS'] = copied_all_hydrants_df['FLSHFRE'].apply(convert_to_days)
print(copied_all_hydrants_df[['FLSHFRE', 'FLSHFRE_DAYS']].head())

print(copied_all_hydrants_df['FLSHFRE_DAYS'].unique())
print(copied_all_hydrants_df['FLSHFRE'].unique())

In [ ]:
copied_all_hydrants_df.to_csv('hydrantcheck.csv', index=False)

### Merging the datasets

In [ ]:
#TODO: Ayan to find the right column to merge the 2 datasets. After merging, delete all the ...KEY columns (identification columns) in the merge df
# Renaming the column so that the name is the same in both datasets
copied_all_hydrants_df.rename(columns={'IMSRO_COMPHY_COMPKEY': 'COMPKEY'}, inplace=True)

In [ ]:
# Merging the DataFrames based on the 'COMPKEY' column - similar to a SQL left outer join
merged_df = pd.merge(copied_all_hydrants_df, copied_flushing_history_df, on='COMPKEY', how='right', indicator=True)

# Removing the COMPKEY column now that we don't need it
merged_df.drop(['COMPKEY'], axis=1, inplace=True)

In [ ]:
merged_df['FLSHFRE_y'].unique()

In [ ]:
# Adjusting incorrectly formatted rows
merged_df['FLSHFRE_y'] = merged_df['FLSHFRE_y'].replace(['DAYS', '2DAYS'], '2 DAYS')

In [ ]:
merged_df['FLSHFRE_y'].unique()

In [ ]:
# Using the FLSHFRE values from the Flushing History dataset to get the frequency in days
merged_df['FLSHFRE_DAYS'] = merged_df['FLSHFRE_y'].apply(convert_to_days)

In [ ]:
# One hot encode the letters again since there are still some blanks ***
merged_df['Letters_B'] = merged_df['Letters_B']

In [ ]:
merged_df.drop('COMMUN_x','COMMUN_y','FLSHFRE_x','FLSHFRE_y','WARDID','SUBAREA','AREA','UNITTYPE')

In [ ]:
# Drop all rows where INSTDATE is blank ***

In [ ]:
# Save the result to a new Excel file
merged_df.to_excel('merged_dataset.xlsx', index=False)

### Feature Selection
Using mutual information

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Separate features and target
x = merged_df.drop('FLSHFRE_DAYS', axis=1)
y = merged_df['FLSHFRE_DAYS']

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify numeric and categorical columns
numeric_features = x.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x.select_dtypes(include=['object', 'category']).columns

# Encode categorical features
le = LabelEncoder()
for feature in categorical_features:
    x[feature] = le.fit_transform(x[feature])

In [ ]:
# Calculate mutual information scores
mi_scores = mutual_info_regression(x, y)

# Create a dataframe of features and their MI scores
mi_scores_df = pd.DataFrame({'feature': x.columns, 'mi_score': mi_scores})
mi_scores_df = mi_scores_df.sort_values('mi_score', ascending=False)

# Print the mutual information scores
print(mi_scores_df)

### Split the data into training and testing sets

In [ ]:
numerical_features = ['AGE', 'FEEDERLEN']
categorical_features = ['Letters_B', 'Letters_C', 'Letters_S', 'LONGKEY', 'PLUGGED', 'NONSCHE', 'DECLREQD', 'HYMARKER', 'MODELNO_AVK', 'MODELNO_MUELLER', 'MODELNO_CLOW']

In [ ]:
# The class to split the data into training and testing sets
from sklearn.model_selection import train_test_split
# Separating the dataset by features (independent variables) and target (dependent variable)
# x = merged_df[]
# y = merged_df['FLSHFRE_DAYS']

In [ ]:
# Separating each of X and Y datasets into training and testing sets
# Fraction of data to be used for testing; the rest is to be used for training
test_size = 0.3
random_state = 42

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)

### Scaling the Data

In [ ]:
# Importing the class to use for scaling the data
from sklearn.preprocessing import MinMaxScaler

# Instantiating a MinMaxScaler object
scaler = MinMaxScaler()
# Fitting the scaler to the training features
scaler.fit(x_train)
# Scaling the training and testing features
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# Training the Model

### Model 1: Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

### Hyperparameter Optimization

In [ ]:
import optuna

### Model 2: XGBoost Regression

### Hyperparameter Optimization

### Model 3: Support Vector Regression

### Hyperparameter Optimization

### Model 4: Random Forest Regression

### Hyperparameter Optimization

# Result Comparison

## Comparing MSE (mean squared error)

## Consider adjusting the train-test split for better results

# Testing the Model